In [6]:
import os

import japanize_matplotlib
import keras_tuner as kt
import lightgbm as lgb
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import tensorflow as tf
import tqdm
from pyspark.sql import SparkSession
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    log_loss,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs,
# please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam

from JapanHorseRaceAnalytics.utilities.base import get_base_dir, read_hive_table

# Set pandas display options
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 200)

# Set seed for reproducibility
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)
random_state = 42

In [2]:
warehouse_dir = f"{get_base_dir()}/spark-warehouse"
postgres_driver_path = f"{get_base_dir()}/jars/postgresql-42.7.1.jar"

spark = (
    SparkSession.builder.appName("20240211_competitors")
    .config("spark.driver.memory", "21g")
    .config("spark.sql.warehouse.dir", warehouse_dir)
    .config("spark.jars", postgres_driver_path)
    .config("spark.executor.extraClassPath", postgres_driver_path)
    .config("spark.driver.extraClassPath", postgres_driver_path)
    .enableHiveSupport()
    .getOrCreate()
)

24/02/24 15:46:39 WARN Utils: Your hostname, Hanks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.40.105 instead (on interface en0)
24/02/24 15:46:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/02/24 15:46:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/24 15:46:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = read_hive_table(
    table_name="features_20240217_v1",
    schema="jhra_curated",
    spark_session=spark,
    # use_cache=False,
)

# drop from data where cat_トラック種別 == "障害"
data = data[(data["cat_トラック種別"] != "障害")]

# This would mess up the number of horses in the race
# drop from data where meta_int_race_horses_異常区分 != '0'
# data = data[(data["meta_int_race_horses_異常区分"] == "0")]

data = data.reset_index(drop=True)
data.head()

{"event": "Read from parquet /Users/hankehly/Projects/JapanHorseRaceAnalytics/data/sql_tables/features_20240217_v1.snappy.parquet to pandas", "level": "info", "timestamp": "2024-02-24T06:46:41.642947Z", "logger": "JapanHorseRaceAnalytics.utilities.base"}


,meta_レースキー,meta_馬番,meta_着順,meta_本賞金,meta_単勝的中,meta_単勝払戻金,meta_複勝的中,meta_複勝払戻金,meta_int_races_レースキー,meta_発走日時,meta_場コード,cat_四半期,cat_距離,cat_事前_馬場状態コード,cat_事前_レース条件_トラック情報_右左,cat_事前_レース条件_トラック情報_内外,cat_事前_レース条件_種別,cat_事前_レース条件_条件,cat_事前_レース条件_記号,cat_事前_レース条件_重量,cat_事前_レース条件_グレード,num_事前_馬場差,cat_実績_馬場状態コード,cat_実績_レース条件_トラック情報_右左,cat_実績_レース条件_トラック情報_内外,cat_実績_レース条件_種別,cat_実績_レース条件_条件,cat_実績_レース条件_記号,cat_実績_レース条件_重量,cat_実績_レース条件_グレード,num_実績_馬場差,num_頭数,cat_トラック種別,cat_馬場状態内,cat_馬場状態中,cat_馬場状態外,num_直線馬場差最内,num_直線馬場差内,num_直線馬場差中,num_直線馬場差外,num_直線馬場差大外,cat_芝種類,cat_草丈,cat_転圧,cat_凍結防止剤,num_中間降水量,meta_int_race_horses_レースキー,meta_int_race_horses_馬番,meta_int_race_horses_血統登録番号,meta_int_race_horses_発走日時,meta_int_race_horses_異常区分,num_事前_ＩＤＭ,cat_事前_脚質,num_事前_単勝オッズ,num_事前_複勝オッズ,cat_事前_馬体,cat_事前_気配コード,cat_事前_上昇度,cat_事前_クラスコード,num_事前_テン指数,num_事前_ペース指数,num_事前_上がり指数,num_実績_ＩＤＭ,cat_実績_脚質,num_実績_単勝オッズ,num_実績_複勝オッズ,cat_実績_馬体,cat_実績_気配コード,cat_実績_上昇度,cat_実績_クラスコード,num_実績_テン指数,num_実績_ペース指数,num_実績_上がり指数,num_負担重量,num_馬体重,num_馬体重増減,cat_性別,cat_トラック種別瞬発戦好走馬,cat_トラック種別消耗戦好走馬,num_一走前不利,num_二走前不利,num_三走前不利,num_一走前着順,num_二走前着順,num_三走前着順,num_四走前着順,num_五走前着順,num_六走前着順,num_騎手指数,num_情報指数,num_オッズ指数,num_パドック指数,num_総合指数,cat_馬具変更情報,cat_脚元情報,cat_見習い区分,cat_オッズ印,cat_パドック印,cat_直前総合印,cat_距離適性,...,num_競争相手平均調教師1位完走,num_競争相手調教師1位完走標準偏差,num_競争相手最高調教師トップ3完走,num_競争相手最低調教師トップ3完走,num_競争相手平均調教師トップ3完走,num_競争相手調教師トップ3完走標準偏差,num_競争相手最高調教師1位完走率,num_競争相手最低調教師1位完走率,num_競争相手平均調教師1位完走率,num_競争相手調教師1位完走率標準偏差,num_競争相手最高調教師トップ3完走率,num_競争相手最低調教師トップ3完走率,num_競争相手平均調教師トップ3完走率,num_競争相手調教師トップ3完走率標準偏差,num_競争相手最高調教師場所レース数,num_競争相手最低調教師場所レース数,num_競争相手平均調教師場所レース数,num_競争相手調教師場所レース数標準偏差,num_競争相手最高調教師場所1位完走,num_競争相手最低調教師場所1位完走,num_競争相手平均調教師場所1位完走,num_競争相手調教師場所1位完走標準偏差,num_競争相手最高調教師場所トップ3完走,num_競争相手最低調教師場所トップ3完走,num_競争相手平均調教師場所トップ3完走,num_競争相手調教師場所トップ3完走標準偏差,num_競争相手最高調教師場所1位完走率,num_競争相手最低調教師場所1位完走率,num_競争相手平均調教師場所1位完走率,num_競争相手調教師場所1位完走率標準偏差,num_競争相手最高調教師場所トップ3完走率,num_競争相手最低調教師場所トップ3完走率,num_競争相手平均調教師場所トップ3完走率,num_競争相手調教師場所トップ3完走率標準偏差,num_競争相手最高調教師本賞金累計,num_競争相手最低調教師本賞金累計,num_競争相手平均調教師本賞金累計,num_競争相手調教師本賞金累計標準偏差,num_競争相手最高調教師1位完走平均賞金,num_競争相手最低調教師1位完走平均賞金,num_競争相手平均調教師1位完走平均賞金,num_競争相手調教師1位完走平均賞金標準偏差,num_競争相手最高調教師レース数平均賞金,num_競争相手最低調教師レース数平均賞金,num_競争相手平均調教師レース数平均賞金,num_競争相手調教師レース数平均賞金標準偏差,num_競争相手平均調教師レース数差,num_競争相手平均調教師1位完走差,num_競争相手平均調教師トップ3完走差,num_競争相手平均調教師1位完走率差,num_競争相手平均調教師トップ3完走率差,num_競争相手平均調教師場所レース数差,num_競争相手平均調教師場所1位完走差,num_競争相手平均調教師場所トップ3完走差,num_競争相手平均調教師場所1位完走率差,num_競争相手平均調教師場所トップ3完走率差,num_競争相手平均調教師本賞金累計差,num_競争相手平均調教師1位完走平均賞金差,num_競争相手平均調教師レース数平均賞金差,meta_int_combinations_レースキー,meta_int_combinations_馬番,num_馬騎手レース数,num_馬騎手1位完走,num_馬騎手1位完走率,num_馬騎手トップ3完走,num_馬騎手トップ3完走率,num_馬騎手初二走,num_馬騎手同騎手,num_馬騎手場所レース数,num_馬騎手場所1位完走,num_馬騎手場所1位完走率,num_馬騎手場所トップ3完走,num_馬騎手場所トップ3完走率,num_馬調教師レース数,num_馬調教師1位完走,num_馬調教師1位完走率,num_馬調教師トップ3完走,num_馬調教師トップ3完走率,num_馬調教師初二走,num_馬調教師同調教師,num_馬調教師場所レース数,num_馬調教師場所1位完走,num_馬調教師場所1位完走率,num_馬調教師場所トップ3完走,num_馬調教師場所トップ3完走率,meta_int_race_weather_レースキー,num_temperature,num_precipitation,num_snowfall,num_snow_depth,num_wind_speed,cat_wind_direction,num_solar_radiation,num_local_air_pressure,num_sea_level_air_pressure,num_relative_humidity,num_vapor_pressure,num_dew_point_temperature,cat_weather,num_visibility
0,01011103,04,6.0,0.0,False,0,False,0,01011103,2001-08-04 01:45:00,01,3,1200,20,1,1,12,A3,102,3,None,NaN,21,1,1,12,A3,102,3,None,-18.0,16,芝,1,1,1,1,1,0,0,0,None,NaN,False,False,NaN,01011103,04,98102049,2001-08-04 01:45:00,0,36.0,好位差し,11.5,2.9,None,None,3,18,-12.4,-21.1,-10.9,31.0,追込,14.8,3.4,普通,2,3,18,-18.9,-30.9,-10.5,550,476.0,14,牡,True,False,NaN,NaN,NaN,7.0,2.0,7.0,NaN,NaN,NaN,0.4,0.4,0.0,1.8,38.6,0,0,0,None,4,4,5,...,34.133333,18.575492,204,32,98.066667,44.395896,0.162376,0.027668,0.075334,0.036725,0.403960,0.120735,0.213261,0.076365,47,3,18.733333,12.390677,8,0,2.066667,2.112397,15,0,4.733333,4.464178,0.333333,0.0,0.094180,0.088041,0.5,0.000000,0.207743,0.136352,231606.0,14187.0,65859.266667,53242.331542,1687.439024,567.857143,913.419366,273.792817,458.625743,49.167979,141.408061,101

In [4]:
X = data
y = data["meta_複勝的中"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=random_state
)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

X_train: (857061, 870)
X_test: (214266, 870)
y_train: (857061,)
y_test: (214266,)


In [9]:
# get all columns that start with "num_"
numerical_features = X_train.filter(regex="^num_", axis=1).columns.tolist()
# get all columns that start with "cat_"
categorical_features = X_train.filter(regex="^cat_", axis=1).columns.tolist()

In [11]:
numerical_features

['num_事前_馬場差',
 'num_実績_馬場差',
 'num_頭数',
 'num_直線馬場差最内',
 'num_直線馬場差内',
 'num_直線馬場差中',
 'num_直線馬場差外',
 'num_直線馬場差大外',
 'num_中間降水量',
 'num_事前_ＩＤＭ',
 'num_事前_単勝オッズ',
 'num_事前_複勝オッズ',
 'num_事前_テン指数',
 'num_事前_ペース指数',
 'num_事前_上がり指数',
 'num_実績_ＩＤＭ',
 'num_実績_単勝オッズ',
 'num_実績_複勝オッズ',
 'num_実績_テン指数',
 'num_実績_ペース指数',
 'num_実績_上がり指数',
 'num_負担重量',
 'num_馬体重',
 'num_馬体重増減',
 'num_一走前不利',
 'num_二走前不利',
 'num_三走前不利',
 'num_一走前着順',
 'num_二走前着順',
 'num_三走前着順',
 'num_四走前着順',
 'num_五走前着順',
 'num_六走前着順',
 'num_騎手指数',
 'num_情報指数',
 'num_オッズ指数',
 'num_パドック指数',
 'num_総合指数',
 'num_ローテーション',
 'num_基準オッズ',
 'num_基準人気順位',
 'num_基準複勝オッズ',
 'num_基準複勝人気順位',
 'num_特定情報◎',
 'num_特定情報○',
 'num_特定情報▲',
 'num_特定情報△',
 'num_特定情報×',
 'num_総合情報◎',
 'num_総合情報○',
 'num_総合情報▲',
 'num_総合情報△',
 'num_総合情報×',
 'num_人気指数',
 'num_調教指数',
 'num_厩舎指数',
 'num_騎手期待連対率',
 'num_激走指数',
 'num_展開予想データ_位置指数',
 'num_展開予想データ_道中順位',
 'num_展開予想データ_道中差',
 'num_展開予想データ_後３Ｆ順位',
 'num_展開予想データ_後３Ｆ差',
 'num_展開予想データ_ゴール順位',
 'num_展開予想データ_ゴール差',
 'n

In [ ]:
numeric_features = X_train.select_dtypes("number").columns.tolist()
categorical_features = X_train.select_dtypes("category").columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            categorical_features,
        ),
    ]
)
model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", lgb.LGBMClassifier(**params)),
    ]
)

params = {
    "boosting_type": "gbdt",
    "class_weight": "balanced",
    "colsample_bytree": 0.8016642153767848,
    "feature_fraction": 0.5578235667548754,
    "lambda_l1": 2.551673582227088,
    "lambda_l2": 1.3506414200964172,
    "learning_rate": 0.02904727910263315,
    "max_depth": 10,
    "min_child_samples": 68,
    "min_child_weight": 7.736782598405014,
    "min_split_gain": 0.0071078853628913415,
    "n_estimators": 861,
    "num_leaves": 121,
    "objective": "binary",
    "reg_alpha": 0.25409327833670503,
    "reg_lambda": 0.4275373164043184,
    "seed": 42,
    "subsample": 0.9179630226670973,
    "verbose": -1,
}

rfe = RFE(estimator=lgb.LGBMClassifier(), step=1)

rfe.fit(X_train, y_train)